In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from  pyspark.sql.window import Window

employees = [
    {"emp_id": 101, "name": "Arun",    "dept": "IT",      "salary": 80000, "experience": 5,  "skills": ["Python", "SQL"],      "status": "Active",   "hire_date": "2021-01-10"},
    {"emp_id": 102, "name": "Meena",   "dept": "HR",      "salary": 55000, "experience": 3,  "skills": ["Excel"],               "status": "Active",   "hire_date": "2022-03-15"},
    {"emp_id": 103, "name": "Kiran",   "dept": "IT",      "salary": 90000, "experience": 7,  "skills": ["Python", "PySpark"],  "status": "Resigned", "hire_date": "2018-10-01"},
    {"emp_id": 104, "name": "Bhavani", "dept": "Finance", "salary": 65000, "experience": 4,  "skills": ["Excel", "SQL"],        "status": "Active",   "hire_date": "2020-07-19"},
    {"emp_id": 105, "name": "Akash",   "dept": "IT",      "salary": 75000, "experience": 2,  "skills": ["Python"],              "status": "Active",   "hire_date": "2023-05-02"},
    
    {"emp_id": 106, "name": "Ravi",    "dept": "Sales",   "salary": 45000, "experience": 1,  "skills": [],                      "status": "Active",   "hire_date": "2024-01-11"},
    {"emp_id": 107, "name": "Divya",   "dept": "HR",      "salary": None,  "experience": 6,  "skills": ["Excel", "PowerBI"],    "status": "Active",   "hire_date": "2019-11-28"},
    {"emp_id": 108, "name": "Suresh",  "dept": "Sales",   "salary": 47000, "experience": 2,  "skills": ["Negotiation"],         "status": "Active",   "hire_date": "2021-06-22"},
    {"emp_id": 109, "name": "Jaya",    "dept": "IT",      "salary": 80000, "experience": 5,  "skills": ["SQL"],                 "status": "Active",   "hire_date": "2020-09-10"},
    {"emp_id": 110, "name": "Vijay",   "dept": "Finance", "salary": 70000, "experience": 8,  "skills": ["Excel"],               "status": "Resigned", "hire_date": "2017-12-01"},

    {"emp_id": 111, "name": "Rohit",   "dept": "IT",      "salary": 65000, "experience": 3,  "skills": ["Python"],              "status": "Active",   "hire_date": "2022-02-17"},
    {"emp_id": 112, "name": "Maya",    "dept": "HR",      "salary": 62000, "experience": 4,  "skills": None,                    "status": "Active",   "hire_date": None},
    {"emp_id": 113, "name": "Goutham", "dept": "Sales",   "salary": 52000, "experience": 3,  "skills": ["Negotiation", "CRM"],  "status": "Active",   "hire_date": "2021-08-05"},
    {"emp_id": 114, "name": "Lavanya", "dept": "IT",      "salary": 90000, "experience": 10, "skills": ["Python", "AWS"],      "status": "Active",   "hire_date": "2015-07-14"},
    {"emp_id": 115, "name": "Kavya",   "dept": "Finance", "salary": 58000, "experience": 1,  "skills": ["Excel"],               "status": "Active",   "hire_date": "2023-01-20"},

    {"emp_id": 116, "name": "Manish",  "dept": "IT",      "salary": None,  "experience": 4,  "skills": ["SQL"],                 "status": "Active",   "hire_date": "2020-10-05"},
    {"emp_id": 117, "name": "Rakesh",  "dept": "Sales",   "salary": 48000, "experience": 2,  "skills": ["CRM"],                 "status": None,       "hire_date": "2022-11-11"},
    {"emp_id": 118, "name": "Anita",   "dept": "HR",      "salary": 56000, "experience": 3,  "skills": ["Excel"],               "status": "Active",   "hire_date": "2023-03-01"},
    {"emp_id": 119, "name": "Tarun",   "dept": "Finance", "salary": 70000, "experience": 7,  "skills": ["SQL", "PowerBI"],      "status": "Active",   "hire_date": "2018-02-25"},
    {"emp_id": 120, "name": "Sanjay",  "dept": "IT",      "salary": 76000, "experience": 5,  "skills": ["Python", "SQL"],       "status": "Active",   "hire_date": "2019-09-09"}
]


spark = SparkSession.builder.appName("practice").getOrCreate()

df = spark.createDataFrame(employees)

In [ ]:
# ✅ Question 1
# Write PySpark code to filter only employees from the IT department whose salary is greater than 75,000.

df.filter((F.col("dept")=='IT') & (F.col("salary") >75000)).show()

In [ ]:
# Question:
# Write a PySpark query to find all employees who joined in or after the year 2021.

df.filter(F.year(F.col("hire_date"))>=2021).show()

In [ ]:
# Find employees who have Python in their skills.
df.filter(F.array_contains("skills","Python")).select('name','skills').show()

In [ ]:
# Find the highest-paid employee in each department using a window function.
# Return: emp_id, name, dept, salary, rank.

win = Window.partitionBy("dept").orderBy(F.col("salary").desc())

df1 = df.withColumn("rank",F.rank().over(win))\
  .withColumn("dense_rank",F.dense_rank().over(win))
df.filter(F.col("rank")==1).select("emp_id","name","dept","salary","rank").show()

In [ ]:
# Write a PySpark query to explode the skills array so that each skill appears in a separate row.
# Return only:
# emp_id, name, skill

df2 = df.withColumn("skill",F.explode("skills"))
df2.select('emp_id','name','skill').show()

In [ ]:
# Write a PySpark query to replace all null salaries with 0.
df.fillna(0,subset=["salary"])

In [ ]:
# Write a PySpark query to count how many employees are present in each department.
# Return columns: dept, emp_count

df3 = df.groupBy("dept").agg(F.count("name").alias("number_of_emp"))
df3.show()

In [ ]:
# Some employees have status = null.
# Write a PySpark query to replace null status with "Unknown" using when and otherwise.

df4 =df.withColumn("status",F.when(F.col("status").isNull(),"Unknown").otherwise(F.col("status")))
df4.filter(F.col("status")=='Unknown').show()

In [ ]:
# Find the average salary of each department, ignoring null salaries.
# Return: dept, avg_salary

df5 = df.filter(F.col("salary").isNotNull())
df5 =df5.groupBy("dept").agg(F.mean("salary").alias("avg_salary"))
df5.select('dept','avg_salary').show()

In [ ]:
# Task: Find the second highest salary in each department.
# Return: dept, emp_id, name, salary.

wind = Window.partitionBy("dept").orderBy(F.col("salary").desc())

df6 = df.withColumn("rank",F.rank().over(wind))
df6.filter(F.col("rank")==2).select('dept','emp_id','name','salary').show()



In [ ]:
df7 = df.withColumn("skill",F.explode("skills"))
unique_skills_count = df7.select("skill").distinct().count()  # count distinct skills
print(unique_skills_count)

In [ ]:
# Group employees by department and collect all employee names into a list for each department.
# Return columns: dept, emp_list.
df.groupBy("dept").agg(F.collect_list('name')).show(truncate=False)


In [ ]:
from pyspark.sql import functions as F

df_fixed = df.withColumn(
    "skills",
    F.when(F.col("skills").isNull(), F.array()).otherwise(F.col("skills"))
)

df_exploded = df_fixed.withColumn("skill", F.explode("skills"))

df_exploded.select("emp_id", "name", "skill").show(truncate=False)


In [ ]:
df = df.withColumn('hire_date',F.to_date(F.col("hire_date"),'yyyy-MM-dd'))

df = df.withColumn("days_with_company",F.date_diff(F.current_date(),'hire_date'))

df.select('emp_id', 'name', 'hire_date', 'days_with_company').show()

In [ ]:
# Find employees whose name starts with 'A' or ends with 'y'.
# Return: emp_id, name, dept.

df.filter((F.col('name').startswith('A')) | (F.col('name').endswith('y')))\
  .select('emp_id','name','dept').show()

In [ ]:
# Task: Find all duplicate employee names and the count of how many times each name appears.
# Return columns: name, count.

from pyspark.sql import functions as F

df.groupBy("name") \
  .agg(F.count("*").alias("count")) \
  .filter(F.col("count") > 1) \
  .show(truncate=False)


In [ ]:
# Sort employees first by department ascending, then salary descending.
# Return columns: emp_id, name, dept, salary.

df1 =df.sort((['dept','salary']),ascending=[1,0])
df1.select('emp_id','name','dept','salary').show(5)

df1 = df.orderBy(F.col("dept").asc(), F.col("salary").desc())
df1.select('emp_id','name','dept','salary').show(5)


In [ ]:
# Find the top 2 highest-paid employees in each department.
# Return columns: dept, emp_id, name, salary.


w = Window.partitionBy("dept").orderBy(F.col("salary").desc())

df.withColumn('rank',F.rank().over(w))\
  .filter(F.col("rank")<=2).select("dept", "emp_id", "name", "salary").show()

In [ ]:
# Replace null salaries with the average salary of their department.
# Return columns: emp_id, name, dept, salary.
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Define window partitioned by department
win = Window.partitionBy("dept")

# Replace null salaries with average salary of the department
df_filled = df.withColumn(
    "salary",
    F.when(
        F.col("salary").isNull(),
        F.round(F.avg("salary").over(win), 2)  # round to 2 decimals (optional)
    ).otherwise(F.col("salary"))
)

# df_filled.select("emp_id", "name", "dept", "salary").show()




from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Define window to get dept-wise average salary
win = Window.partitionBy("dept")

df_filled = df.withColumn(
    "salary",
    F.coalesce(F.col("salary"), F.round(F.avg("salary").over(win),2))
)

df_filled.select("emp_id", "name", "dept", "salary").show()


In [ ]:
df = df.withColumn('skill',F.explode('skills'))
df.groupBy('skill').agg(F.count('name').alias('emp_count')).show()

In [ ]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

sales_data = [
    {"txn_id": 1, "cust_id": 101, "amount": 500,  "status": "success", "txn_date": "2024-01-01"},
    {"txn_id": 2, "cust_id": 102, "amount": None, "status": "success", "txn_date": "2024-01-01"},
    {"txn_id": 3, "cust_id": 101, "amount": 200,  "status": "failed",  "txn_date": "2024-01-03"},
    {"txn_id": 4, "cust_id": 103, "amount": 800,  "status": "success", "txn_date": "2024-01-04"},
    {"txn_id": 5, "cust_id": 104, "amount": None, "status": "failed",  "txn_date": "2024-01-05"}
]

df = spark.createDataFrame(sales_data)

# Replace null amount with 0
df = df.fillna(0,subset=['amount'])

# Convert txn_date into proper date type
df = df.withColumn('txn_date',to_date(col('txn_date'),'yyyy-MM-dd'))
# df = df.withColumn('hire_date',F.to_date(F.col("hire_date"),'yyyy-MM-dd'))

# Add a new column is_success →
# 1 if status = "success"
#  ->0 otherwise
df = df.withColumn("is_success",when(col('status')=='success',1).otherwise(0))

# Filter only transactions where amount > 0
df = df.filter(col('amount')>0)

# Final output columns:
# txn_id, cust_id, amount, txn_date, is_success
df.select('txn_id', 'cust_id', 'amount', 'txn_date', 'is_success').show()


In [118]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.window import Window

order_data = [
    {"order_id": 1, "cust_id": 101, "amount": 500, "category": "Electronics", "order_date": "2024-01-01"},
    {"order_id": 2, "cust_id": 101, "amount": 300, "category": "Grocery",     "order_date": "2024-01-03"},
    {"order_id": 3, "cust_id": 102, "amount": 900, "category": "Electronics", "order_date": "2024-01-05"},
    {"order_id": 4, "cust_id": 103, "amount": None,"category": "Grocery",     "order_date": "2024-01-06"},
    {"order_id": 5, "cust_id": 103, "amount": 600, "category": "Fashion",     "order_date": "2024-01-07"},
    {"order_id": 6, "cust_id": 101, "amount": 200, "category": "Fashion",     "order_date": "2024-01-09"}
]

spark = SparkSession.builder.appName("interview").getOrCreate()

df = spark.createDataFrame(order_data)

# 1️⃣ Replace null amount with the average amount of that category

win = Window.partitionBy('category')
df = df.withColumn("amount",coalesce('amount',avg('amount').over(win)))
df.select('order_id', 'cust_id', 'amount', 'category', 'order_date').show()


# 2️⃣ Find total amount per customer
df.groupBy('cust_id').agg(sum('amount').alias("total amount")).show()

# 3️⃣ For each category, find the highest amount order
win = Window.partitionBy('category').orderBy(col('amount').desc())

df.withColumn("dense_rank",dense_rank().over(win))\
  .filter(col("dense_rank")==1)\
  .select('category',"amount").show()

# 4️⃣ Add a new column days_since_order using current_date
df.withColumn("days_since_order",current_date()).show()



+--------+-------+------+-----------+----------+
|order_id|cust_id|amount|   category|order_date|
+--------+-------+------+-----------+----------+
|       1|    101| 500.0|Electronics|2024-01-01|
|       3|    102| 900.0|Electronics|2024-01-05|
|       5|    103| 600.0|    Fashion|2024-01-07|
|       6|    101| 200.0|    Fashion|2024-01-09|
|       2|    101| 300.0|    Grocery|2024-01-03|
|       4|    103| 300.0|    Grocery|2024-01-06|
+--------+-------+------+-----------+----------+

+-------+------------+
|cust_id|total amount|
+-------+------------+
|    103|       900.0|
|    101|      1000.0|
|    102|       900.0|
+-------+------------+

+-----------+------+
|   category|amount|
+-----------+------+
|Electronics| 900.0|
|    Fashion| 600.0|
|    Grocery| 300.0|
|    Grocery| 300.0|
+-----------+------+

+------+-----------+-------+----------+--------+----------------+
|amount|   category|cust_id|order_date|order_id|days_since_order|
+------+-----------+-------+----------+-------